In [1]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch
device = "cuda" # the device to load the model onto

model_name = "Qwen/Qwen1.5-MoE-A2.7B"

# 4-bit QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 # QLoRA = 4-bit base model
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",         # QLoRA uses nf4
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/668M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

In [4]:
model.eval()
inputs = tokenizer("Hello!", return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(out[0], skip_special_tokens=True))

Hello! I'm a 20 year old girl from the Netherlands. I'm a student and I'm


In [42]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from peft import IA3Config, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

config = LoraConfig(r = 4,
                    lora_alpha=4,
                    target_modules = ["gate", "q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
                    lora_dropout=0.1
                    )

lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()

/workspace/.venv/lib/python3.10/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/workspace/.venv/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 63,706,752 || all params: 14,379,490,944 || trainable%: 0.4430


In [7]:
from datasets import load_dataset

# dataset = load_dataset("Na0s/sft-ready-Text-Generation-Augmented-Data", split="train")

In [4]:
# print(dataset)

In [5]:
# def tokenize(batch):
#     texts = []
#     for prompt, completion in zip(batch["prompt"], batch["completion"]):
#         text = tokenizer.apply_chat_template(
#             [
#                 {"role": "user", "content": prompt},
#                 {"role": "assistant", "content": completion},
#             ],
#             tokenize=False,
#             add_generation_prompt=False,
#         )

#         if isinstance(text, list):
#             text = "".join(text)

#         texts.append(text)

#     return tokenizer(
#         texts,
#         truncation=True,
#         padding=False,
#         max_length=2048
#     )

# batch = dataset.select(range(3))
# tokens = tokenize(batch)
# tokenizer.decode(tokens["input_ids"][0])


In [6]:
# tokenized_dataset = dataset.map(
#     tokenize,
#     batched=True,
#     num_proc=75,
#     remove_columns=["prompt", "completion"],
# )

# print(tokenized_dataset[0])

In [8]:
tokenized_dataset = load_dataset("kaaiiii/tokenized_data_for_MoE", split="train")

README.md:   0%|          | 0.00/331 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

data/train-00000-of-00026.parquet:   0%|          | 0.00/212M [00:00<?, ?B/s]

data/train-00001-of-00026.parquet:   0%|          | 0.00/214M [00:00<?, ?B/s]

...

data/train-00023-of-00026.parquet:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

data/train-00024-of-00026.parquet:   0%|          | 0.00/46.6M [00:00<?, ?B/s]

data/train-00025-of-00026.parquet:   0%|          | 0.00/64.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [29]:
# print(tokenized_dataset[0:5])
# tokenized_dataset.save_to_disk("./tokenized_dataset")

{'input_ids': [[151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 15, 13, 15, 15, 17, 284, 220, 16, 15, 15, 15, 715, 16, 284, 856, 30, 151645, 198, 151644, 77091, 198, 1249, 1477, 279, 897, 315, 856, 11, 582, 646, 738, 705, 264, 21117, 1667, 279, 2661, 1995, 1447, 15, 13, 15, 15, 17, 14, 16, 15, 15, 15, 284, 220, 16, 10776, 271, 1249, 11625, 369, 856, 11, 582, 646, 5312, 30270, 1447, 15, 13, 15, 15, 17, 353, 856, 284, 220, 16, 15, 15, 15, 353, 220, 16, 271, 15, 13, 15, 15, 17, 87, 284, 220, 16, 15, 15, 15, 271, 12509, 6577, 2176, 11067, 553, 220, 15, 13, 15, 15, 17, 1447, 87, 284, 220, 16, 15, 15, 15, 608, 220, 15, 13, 15, 15, 17, 271, 87, 284, 220, 20, 15, 15, 11, 15, 15, 15, 271, 54815, 11, 220, 16, 374, 6144, 311, 220, 20, 15, 15, 11, 15, 15, 15, 304, 419, 21117, 13, 151645, 198], [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 15, 25, 15, 15, 198, 12555, 594, 2087, 389, 7598, 432, 594, 19122, 62, 16, 504, 198, 15, 25, 1

Saving the dataset (0/26 shards):   0%|          | 0/7667416 [00:00<?, ? examples/s]

In [15]:

tokenizer.decode(tokenized_dataset["train"]["input_ids"][0])

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n0.002 = 1000 \n1 = x?<|im_end|>\n<|im_start|>assistant\nTo find the value of x, we can set up a proportion using the given information:\n\n0.002/1000 = 1/x\n\nTo solve for x, we can cross multiply:\n\n0.002 * x = 1000 * 1\n\n0.002x = 1000\n\nDividing both sides by 0.002:\n\nx = 1000 / 0.002\n\nx = 500,000\n\nTherefore, 1 is equal to 500,000 in this proportion.<|im_end|>\n'

In [24]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print(lora_model.hf_device_map)

CUDA available: True
Device count: 1
Current device: 0
{'': 0}


In [39]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [ ]:
lora_model = PeftModel.from_pretrained(
    model,
    "Qwen1.5_MoE_lora_model_100steps"
)

In [44]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

train_dataset = tokenized_dataset["train"]

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM
)

training_args = TrainingArguments(
    output_dir="./outputs_2",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=500,               # try a small number first
    learning_rate=1e-4,
    weight_decay=0.03,
    lr_scheduler_type="cosine",
    warmup_steps=5,
    bf16=True,

    logging_steps=1,
    logging_strategy="steps",
    logging_first_step=True,
    report_to="none",      # prevents WandB from hijacking logs
    disable_tqdm=False    # ensure progress bar is visible

    save_strategy="steps",
    save_steps=100,
    save_total_limit=10,
    save_safetensors=True,
)

trainer = Trainer(
    model=lora_model,          # your LoRA-wrapped model
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

trainer.model.save_pretrained("Qwen1.5_MoE_lora_model_600steps")

PyTorch: setting up devices
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
***** Running training *****
  Num examples = 7,667,416
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 100
  Number of trainable parameters = 63,706,752
/workspace/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.948700
2,2.059900
3,1.830700
4,2.581200
5,1.727300
6,1.027500
7,1.917700
8,1.981200
9,2.194300
10,1.945800


Saving model checkpoint to ./outputs/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/config.json
Model config Qwen2MoeConfig {
  "architectures": [
    "Qwen2MoeForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "decoder_sparse_step": 1,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 8192,
  "max_window_layers": 21,
  "mlp_only_layers": [],
  "model_type": "qwen2_moe",
  "moe_intermediate_size": 1408,
  "norm_topk_prob": false,
  "num_attention_heads": 16,
  "num_experts": 60,
  "num_experts_per_tok": 4,
  "num_hidden_layers": 24,
  "num_key_value_heads": 16,
  "output_router_logits": false,
  "qkv_bias": true,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "router_aux_

In [45]:
trainer.model.push_to_hub("kaaiiii/Qwen1.5b_MoE_LoRA")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/config.json
Model config Qwen2MoeConfig {
  "architectures": [
    "Qwen2MoeForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "decoder_sparse_step": 1,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 8192,
  "max_window_layers": 21,
  "mlp_only_layers": [],
  "model_type": "qwen2_moe",
  "moe_intermediate_size": 1408,
  "norm_topk_prob": false,
  "num_attention_heads": 16,
  "num_experts": 60,
  "num_experts_per_tok": 4,
  "num_hidden_layers": 24,
  "num_key_value_heads": 16,
  "output_router_logits": false,
  "qkv_bias": true,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "router_aux_loss_coef": 0.001,
  "shared_expert_intermediate_siz

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/kaaiiii/Qwen1.5b_MoE_LoRA/commit/dc41c10c94e28e6281b1ee7a0c9e6c164b6a59e7', commit_message='Upload model', commit_description='', oid='dc41c10c94e28e6281b1ee7a0c9e6c164b6a59e7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kaaiiii/Qwen1.5b_MoE_LoRA', endpoint='https://huggingface.co', repo_type='model', repo_id='kaaiiii/Qwen1.5b_MoE_LoRA'), pr_revision=None, pr_num=None)

In [35]:
# from trl import SFTConfig
# import logging
# logging.basicConfig(level=logging.INFO)

# from transformers import logging as hf_logging
# hf_logging.set_verbosity_info()

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-MoE-A2.7B", use_fast=True)
# print(lora_model.device)

# trainer = SFTTrainer(
#     model = lora_model,
#     train_dataset = tokenized_dataset["train"],
#     # processing_class = tokenizer,
#     args = SFTConfig(
#         dataset_kwargs={"skip_prepare_dataset": True},
#         per_device_train_batch_size = 8,
#         gradient_accumulation_steps = 1,
#         packing = True,
#         group_by_length = True,
#         warmup_steps = 5,
#         bf16 = True,
#         max_steps=500,
#         learning_rate = 1e-4,
#         optim = "adamw_8bit",
#         weight_decay = 0.03,
#         lr_scheduler_type = "cosine",
#         seed = 3407,
#         eval_strategy="no",
#         do_eval=False,
#         output_dir = "./outputs",
#         push_to_hub=True,
#         remove_unused_columns=False,
#         logging_steps = 5,
#         disable_tqdm=False,
#         report_to="none"
#     )
# )

# torch.cuda.empty_cache()

# trainer.train()

# trainer.model.save_pretrained("Qwen1.5_MoE_lora_model")

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
PyTorch: setting up devices


cuda:0


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-MoE-A2.7B/snapshots/1a758c50ecb6350748b9ce0a99d2352fd9fc11c9/config.json
Model config Qwen2MoeConfig {
  "architectures": [
    "Qwen2MoeForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "decoder_sparse_step": 1,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 8192,
  "max_window_layers": 21,
  "mlp_only_layers": [],
  "model_type": "qwen2_moe",
  "moe_intermediate_size": 1408,
  "norm_topk_prob": false,
  "num_attention_heads": 16,
  "num_experts": 60,
  "num_experts_per_tok": 4,
  "num_hidden_layers": 24,
  "num_key_value_heads": 16,
  "output_router_logits": false,
  "qkv_bias": true,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "router_aux_loss_coef": 0.001,
  "shared_expert_intermediate_siz

KeyboardInterrupt: 